In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=16, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output

# Load embeddings
graph_dir = "graph_embeddings"
years = list(range(2000, 2024))
embeddings = {year: torch.load(os.path.join(graph_dir, f"embeddings_{year}.pt")) for year in years}

# Prepare data for prediction
window_size = 10
num_nodes, features = embeddings[2000].shape
input_dim = num_nodes * features

lstm_model = LSTMModel(input_dim)
lstm_model.load_state_dict(torch.load("lstm_model.pth"))
lstm_model.eval()

# Predict for future years (2024-2027)
predicted_years = range(2024, 2028)
future_embeddings = {}

past_10_years = [embeddings[year] for year in range(2014, 2024)]
input_seq = torch.stack(past_10_years).unsqueeze(0).reshape(1, window_size, input_dim)

for year in predicted_years:
    predicted_embeddings = lstm_model(input_seq).squeeze(0).reshape(num_nodes, features)
    future_embeddings[year] = predicted_embeddings.detach()
    torch.save(predicted_embeddings, f"{graph_dir}/embeddings_{year}.pt")
    print(f"Predictions for {year} saved!")

    # Update input sequence by appending the predicted year and removing the oldest year
    input_seq = torch.cat([input_seq[:, 1:, :], predicted_embeddings.unsqueeze(0).reshape(1, 1, input_dim)], dim=1)

print("Future predictions completed and saved.")


Predictions for 2024 saved!
Predictions for 2025 saved!
Predictions for 2026 saved!
Predictions for 2027 saved!
Future predictions completed and saved.
